In [1]:
import pyvisa,time
from decode_scope_data import LecroyScopeData
import numpy as np
from scipy.integrate import simpson

In [2]:
#Modify the following global variables according to the model
DSO_RSC = "TCPIP0::10.195.48.42::inst0::INSTR"

rm = pyvisa.ResourceManager('@py')
dso = rm.open_resource(DSO_RSC)
dso.timeout = 30000 #default value is 2000(2s)
dso.chunk_size = 20*1024*1024 #default value is 20*1024(20k bytes)

print(dso.query("*IDN?").strip())

Lecory,T3DSO2502A,T0101C21490989,5.0.1.3.9R6


In [ ]:
dso.write("TRIG:MODE SINGLE")
i = 0

with open("./SiPMT_data.txt","w", encoding="utf-8") as f:
    while True:
        while dso.query("TRIGGER:STAT?").strip() != 'Stop':
            time.sleep(0.25)
            pass
        
        dso.write("WAV:SOUR C4")
        dso.write("WAV:PREamble?")
        recv_preamble = dso.read_raw()

        dso.write("WAV:DATA?")
        recv_data = dso.read_raw()

        waveform = LecroyScopeData(recv_preamble,recv_data)
        voltage = waveform.y
        t = waveform.x

        m = min(voltage)
        index_min = np.where(m == voltage)[0][0]
        ground = np.average(voltage[0:index_min-10])
        y = voltage-ground
        integral = simpson(y,t)

        out = str(m) + ',' + str(integral)
        f.write(out + '\n')
        i += 1
        print('Counts:', i)

        dso.write("TRIG:MODE SINGLE")